In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import mean_absolute_error
import os


In [10]:
def mae_loss_func(weights):
    
    ''' scipy minimize will pass the weights as a numpy array '''
    final_prediction = 0
    
    for weight, prediction in zip(weights, predictions):
            final_prediction += weight*prediction
    return mean_absolute_error(y_train1, final_prediction)


In [3]:
xgb_tr1 = pd.read_csv('Good_preds/XGB_LGBM/train2_LGBM_30.11_10folds_0.003eta_interactions.csv')
xgb_tr2 = pd.read_csv('Good_preds/XGB_LGBM/train2_XGBM_2&3_30.11_10folds_0.01eta_interactions.csv')
xgb_tr3 = pd.read_csv('Good_preds/XGB_LGBM/train3_preds_XGB2&3+LGBM3_interactions_eta0.03.csv')
xgb_tr4 = pd.read_csv('Good_preds/XGB_LGBM/train1_XGB4_eta0.01_2.12.csv')
xgb_tr5 = pd.read_csv('Good_preds/XGB_LGBM/train3_preds_no_interactions_eta0.01_oldparams.csv')
xgb_tr6 = pd.read_csv('Good_preds/XGB_LGBM/train1_XGBL2_stack_log+210_4.12.csv')
xgb_tr7 = pd.read_csv('Good_preds/XGB_LGBM/train1_XGB_Alikernel.csv')['0']


nn_tr1 = pd.read_csv('Good_preds/NN/train1_kernelbagging_log+210.csv')
nn_tr2 = pd.read_csv('Good_preds/NN/train1_nn3_10folds_10bags_loss+1.csv')
nn_tr3 = pd.read_csv('Good_preds/NN/train1_kernelbagging_nonlog.csv')
nn_tr4 = pd.read_csv('Good_preds/NN/train1_nn3_10fold5bags_log+210.csv')
nn_tr5 = pd.read_csv('Good_preds/NN/train1_nn3_10folds_10bags_loss+1.csv')
nn_tr6 = pd.read_csv('Good_preds/NN/train1_nn4_10folds_10bags_loss+1.csv')
nn_tr7 = pd.read_csv('Good_preds/NN/train1_nn1_log+210.csv')
nn_tr8 = pd.read_csv('Good_preds/NN/train1_NNL2_raw_10folds3bags_6.12.csv')


sklearn_knn = pd.read_csv('Good_preds/Sklearn/train_knn_sklearn_algos_4.12.csv')
sklearn_sparse = pd.read_csv('Good_preds/Sklearn/train_sparse_sklearn_algos_5.12.csv')
etrf_tr = pd.read_csv('Good_preds/Sklearn/train2_ET&RF_Faron_6.12.csv')
vw_tr = pd.read_csv('Good_preds/Sklearn/train1_VW_interactions_7.12.csv')
vw_quant_tr = pd.read_csv('Good_preds/Sklearn/train1_VW_Quantile_interactions_7.12.csv')
fm_mcmc_tr = pd.read_csv('Good_preds/Sklearn/train1_FFM-MCMC_interactions_7.12.csv')
rgf_tr = pd.read_csv('Good_preds/Sklearn/train1_RGF_13500_5fold_7.12.csv')



nn_tr1.set_index('id', inplace = True)
nn_tr1.sort_index(inplace = True)
nn_tr1.reset_index(inplace = True)

nn_tr3.set_index('id', inplace = True)
nn_tr3.sort_index(inplace = True)
nn_tr3.reset_index(inplace = True)

nn_tr1 = nn_tr1['loss']
nn_tr3 = nn_tr3['loss']

preds_tr = pd.concat([xgb_tr1, xgb_tr2, xgb_tr3, xgb_tr4, xgb_tr5, xgb_tr7,
                      nn_tr1, nn_tr3, nn_tr2, nn_tr4, nn_tr5, nn_tr6, nn_tr7,
                     sklearn_knn, sklearn_sparse, etrf_tr, 
                     vw_tr, vw_quant_tr, fm_mcmc_tr, rgf_tr], 1)

preds_tr.columns = xrange(preds_tr.shape[1])



X_train1 = preds_tr
y_train1 = pd.read_csv('../../train.csv')['loss']

import gc
del preds_tr
gc.collect()

7

In [6]:
preds_tr = X_train1.iloc[:, [11, 17, 19, 20, 1, 21, 22,
                             13, 24, 25, 26, 27, 31, 3, 32, 12, 34, 35, 7, 36]]
preds_tr.columns = xrange(preds_tr.shape[1])

In [7]:
predictions = []

for i in range(preds_tr.shape[1]):
    predictions.append(preds_tr[i])

In [8]:

starting_values = np.random.uniform(size = len(predictions))
cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
bounds = [(0,1)] * len(predictions)


In [11]:
res = minimize(mae_loss_func, 
           starting_values, 
           method = 'L-BFGS-B', 
           bounds = bounds, 
           constraints = cons,
           options={'maxiter': 10000})

best_score = res['fun']
weights = res['x']

print('Ensemble Score: {}'.format(best_score))
print('Best Weights: {}'.format(weights))


Ensemble Score: 1119.2127729
Best Weights: [ 0.0105157   0.28526925  0.          0.          0.11016722  0.          0.
  0.01838001  0.          0.          0.          0.          0.          0.2597759
  0.          0.1299494   0.          0.          0.19759863  0.        ]
